##Assignment_14

##a) 아래 예제 코드를 이용해 텍스트 입력의 숫자 변환 과정을 체크한다

##b) testset의 임의 입력을 LongTensor로 변환해 학습 완료된 모델에 입력해보고, 결과가 어떠한지 체크한다 (하단의 출력 결과를 참조)

In [ ]:
import os
import torch
import torch.nn as nn
from torchtext.legacy import data, datasets 
from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
batch_size = 64
learning_rate = 0.001
training_epochs = 5

Mounted at /content/drive
cuda


In [ ]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(trainset, min_freq=5)
LABEL.build_vocab(trainset)

trainset, valset = trainset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=batch_size,
        shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = 2

print("[TrainSet]: %d [ValSet]: %d [TestSet]: %d [Vocab]: %d [Classes] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:02<00:00, 42.0MB/s]


[TrainSet]: 20000 [ValSet]: 5000 [TestSet]: 25000 [Vocab]: 46159 [Classes] 2


In [ ]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        self.n_layers = n_layers

        self.embed = nn.Embedding(n_vocab, embed_dim)

        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)

        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        
        x = self.embed(x)
        x, _ = self.gru(x)


        h_t = x[:,-1,:]

        self.dropout(h_t)

        out = self.out(h_t)  # [b, h] -> [b, o]
        return out

## a) 숫자 변환 과정 체크

In [ ]:
input_text = testset[3].text
print(input_text)
print(TEXT.vocab[input_text[10]])

['musical', 'bios', 'are', 'all', 'cut', 'of', 'the', 'same', 'cloth.', 'hopeful', 'struggles,', 'succeeds', 'and', 'finally', 'wins', 'the', 'girl,', 'but', 'this', 'one', '-', 'a', 'life', 'of', 'irish', 'tenor,', 'chauncey', 'olcott/jack', 'chancellor', '-', 'has', 'more', 'going', 'for', 'it', 'than', 'the', 'usual', 'trappings.', 'it', 'has', 'great', 'charm', 'and', 'great', 'sincerity', 'and', 'is', 'played', 'beautifully', 'by', 'all', 'concerned.', 'dennis', 'morgan', 'is', 'fine', 'in', 'the', 'lead', 'as', 'is', 'arlene', 'dahl', 'as', 'his', 'love', 'interest.', 'andrea', "king's", 'supporting', 'performance', 'as', 'lillian', 'russell', 'is', 'far', 'better', 'than', 'alice', "faye's", 'leading', 'bio', 'performance', 'in', 'the', 'film', 'dedicated', 'to', 'her', 'career.', 'william', 'frawley', 'is', 'touching', 'as', 'the', 'aging', 'tenor', 'champion,', 'william', 'scanlon,', 'and', 'sara', 'allgood', 'is', 'lovely', 'as', "olcott's", 'mother.', 'george', 'tobias,', 'b

In [ ]:
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(training_epochs):
  avg_cost = 0
  for batch in train_iter:
    X, Y = batch.text.to(device), batch.label.to(device)
    Y.data.sub_(1)
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    avg_cost += cost / batch_size
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

torch.save(model.state_dict(), '/content/drive/MyDrive/model_s1.pt')

[Epoch:    1] cost = 3.41757894
[Epoch:    2] cost = 3.3985014
[Epoch:    3] cost = 3.39083004
[Epoch:    4] cost = 3.39621067
[Epoch:    5] cost = 2.61155677
Learning Finished!


In [ ]:
for batch in train_iter:
  x = batch.text
  print(x,x.shape, "\n \n")

tensor([[    2, 20816,    14,  ...,     1,     1,     1],
        [  145,    23,     3,  ...,     1,     1,     1],
        [   50,     9,   273,  ...,     1,     1,     1],
        ...,
        [ 3299, 17235,     7,  ...,     1,     1,     1],
        [  108,    13,   165,  ...,     1,     1,     1],
        [ 2060,     7,    23,  ...,     1,     1,     1]]) torch.Size([64, 732]) 
 

tensor([[  12,   14,  356,  ...,    1,    1,    1],
        [  48,   32, 4305,  ...,    1,    1,    1],
        [   9,   62,    3,  ...,    1,    1,    1],
        ...,
        [   9,   86,   98,  ...,    1,    1,    1],
        [   2,  225,  490,  ...,    1,    1,    1],
        [  10,  616,    7,  ...,    1,    1,    1]]) torch.Size([64, 989]) 
 

tensor([[   82,     5,   364,  ...,     1,     1,     1],
        [ 2405,  1809,    46,  ...,     1,     1,     1],
        [  347,    87,   381,  ...,     1,     1,     1],
        ...,
        [   10,     7,    30,  ...,     1,     1,     1],
        [ 1551,

In [ ]:
model_new = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
model_new.load_state_dict(torch.load('/content/drive/MyDrive/model_s1.pt'))

corrects = 0
for batch in val_iter:
  x,y = batch.text.to(device), batch.label.to(device)
  y.data.sub_(1)
  hypothesis = model_new(x)
  corrects += (hypothesis.max(1)[1].view(y.size()).data == y.data).sum() 

print('accuracy = ', corrects/len(val_iter.dataset)*100.0)

accuracy =  tensor(81.3800, device='cuda:0')


##b) LongTensor 변환

In [ ]:
model_new = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
model_new.load_state_dict(torch.load('/content/drive/MyDrive/model_s1.pt'))

Arbitrary_text = testset[4].text
LongTensor = torch.LongTensor
#Arbitrary_tensor_list = []
input_batch = []


for text in Arbitrary_text:
  Vocab = TEXT.vocab[text]
  textTOtensor = torch.LongTensor(Vocab).to(device)
  input_batch.append(textTOtensor)
LongTensor=torch.cat(input_batch)
hypothesis = model_new(LongTensor)
  
  #hypothesis = model_new(input_batch)
  #print(textTOtensor)
  #print("\n\n\n")

  #Arbitrary_tensor_list.append(textTOtensor)
  #ANS = model(textTOtensor)
  #LongTensor = torch.cat([LongTensor,textTOtensor], dim = 2)

#model_new(LongTensor)

#Arbitrary_tensor = torch.stack(Arbitrary_tensor_list)
#ANS = model_new(Arbitrary_tensor)

RuntimeError: ignored